<b>Data mining Project - 2021/22</b><br/>
<span>
<b>Authors:</b> Mariagiovanna Rotundo (560765), Nunzio Lopardo (600005)</a> and Renato Eschini (203021)<br/>
<b>Group:</b>3<br/>
<b>Release date:</b> 26/12/2021
</span>

# Task 2: Data Clustering

This workbook contains a clustering analysis on the tennis dataset.
This dataset was derived from the "tennis_matches.csv" dataset by deriving player information from the matches played.

A new "performance" dataset was also derived in which the data was reprocessed to use only essential and important information for a player-related clustering analysis, eg. only players who have played more than a certain number of games will be considered, only numerical attributes, percentages strictly greater than zero, etc.

**Importing libraries**

In [ ]:
import math
from datetime import date
import re
import collections
import os

import numpy as np

import matplotlib.pyplot as plt

import scipy.stats as stats
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import AgglomerativeClustering

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, pairwise_distances
from sklearn.neighbors import NearestNeighbors

import pandas as pd

from tqdm.notebook import tqdm

from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.gmeans import gmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster import cluster_visualizer, cluster_visualizer_multidim

import seaborn as sns

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

**Loading the dataset**

Load "players.csv"

In [ ]:
# load of the data
DATASET_DIR = "dataset" + os.path.sep
#index_col=False say to not use the first column as ID
df_players = pd.read_csv('players.csv', sep=',', index_col=0) 
df_players.head()

Print info

In [ ]:
df_players.info()

## Perfomances dataset

We analyze the distribution of players by number of matches played

In [ ]:
sns.histplot(data=df_players['nmatch'], bins="auto", binrange=(10,400), color="lightgreen", kde=True)

In [ ]:
sns.histplot(data=df_players['best_rank'], bins="auto", color="lightgreen", kde=True)

We create the performance dataset by taking only some attributes and the players who have at least a fixed number of n_match games (for example at least 100 games played)

In [ ]:
n_match = 100

df_performances_org = df_players[
    (df_players['best_rank']>0) & 
    (df_players['best_rank_points']>=0) & 
    (df_players['tot_minutes']>0) & 
    (df_players['ace']>=0) & 
    (df_players['bpS']>=0)][[
'best_rank', 
'best_rank_points',                            
'tot_minutes',
'sv1st_win', 
'sv2nd_win', 
'df', 
'ace', 
'bpS', 
'nmatch', 
'best_of_3_match', 
'best_of_3_wins', 
'best_of_5_match', 
'best_of_5_wins', 
'wmatch',
'lmatch', 
'w_tourney',
'n_tourney']]
df_performances = df_performances_org.loc[df_performances_org['nmatch'] > n_match]

df_performances

## Elimination of highly correlated features

We begin by examining the correlations between the attributes of the dataset to be clustered in order to identify the highly correlated couples. Dropping redundant attributes benefits the analysis by reducing the dimensionality of the dataset and rising the influence that more useful feature could have on the whole clustering process.

With such aim in mind, we fix a maximum threshold value in order to identify highly correlated features and subsequently drop them.

In [ ]:
plt.figure(figsize = (15,6))
sns.heatmap( df_performances.corr(), annot=True)

In [ ]:
corr_threshold = 0.9
print("Att. A\t\t\tAtt. B\t\t\tCorr(A,B)")
for i in range(0, len(df_performances.columns)):
    for j in range(i+1, len(df_performances.columns)):
        corr = df_performances[df_performances.columns[i]].corr(df_performances[df_performances.columns[j]])
        if  corr > corr_threshold:
            print(df_performances.columns[i] + "\t\t\t" + df_performances.columns[j] + "\t\t\t" + '{:.4f}'.format(corr))

In [ ]:
#   
corr_columns = ['best_of_3_match', 'best_of_5_match', 'sv2nd_win', 'tot_minutes']
tmp = df_performances.drop(corr_columns, axis=1, inplace=False)
df_performances = tmp
df_performances

## K-Means

## Identification of the best value of k 
The **K** parameter represents the number of clusters into which we want to divide the dataset. <br />
There are different methods to calculate the value of K. <br />
Below we use the **Elbow** method with a ready-made library and by simulating clustering with different k and then measuring the results obtained by **Sum of Square Errors (SSE)** and by **Silhouette**, then measuring the results.

#### KElbowVisualizer with 3 different metrics

In [ ]:
# Instantiate the clustering model and visualizer with 
# mean sum of squared distances to centers (distortion) metric

model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,16), metric="distortion")

visualizer.fit(df_performances)        # Fit the data to the visualizer
visualizer.show()                      # Finalize and render the figure

In [ ]:
# Instantiate the clustering model and visualizer with 
# mean ratio of intra-cluster and nearest-cluster distance (silhouette)

model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,16), metric="silhouette")

visualizer.fit(df_performances)        # Fit the data to the visualizer
visualizer.show()                      # Finalize and render the figure

In [ ]:
# Instantiate the clustering model and visualizer with 
# ratio of within to between cluster dispersion (calinski_harabasz)

model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,16), metric="calinski_harabasz")

visualizer.fit(df_performances)        # Fit the data to the visualizer
visualizer.show()                      # Finalize and render the figure

#### Simulate different clustering with different K and evaluate by hand silhouette_score and sum of standard error

In [ ]:
sse_list = list()
sil_list = list()
k_cols = []

max_k = 16
for k in tqdm(range(2, max_k + 1), total=max_k - 1, desc="Iterating over possible K values"):
    kmeans_iter = KMeans(n_clusters=k, n_init=10, max_iter=100)
    kmeans_iter.fit(df_performances)        
    sil_list.append(silhouette_score(df_performances, kmeans_iter.labels_))
    sse = int(kmeans_iter.inertia_)
    sse_list.append(sse)
    k_cols.append(str(k))
    
store_results=pd.DataFrame()
store_results['K'] = k_cols
store_results['SSE'] = sse_list
store_results['Silhouette'] = sil_list
store_results.set_index(['K'], inplace=True)
store_results

In [ ]:
# plot indicators
fig, axs = plt.subplots(2,1,figsize=(15,25))
axs[0].plot(range(2, len(sse_list) + 2), sse_list)
axs[0].set_ylabel('SSE', fontsize=22)
axs[0].set_xlabel('K', fontsize=22)
axs[0].tick_params(axis='both', which='major', labelsize=10)

axs[1].plot(range(2, len(sil_list) + 2), sil_list)
axs[1].set_ylabel('Silhouette', fontsize=22)
axs[1].set_xlabel('K', fontsize=22)
axs[1].tick_params(axis='both', which='major', labelsize=10)

**We can assume that this is the best value of k**

In [ ]:
n_clusters = 4

**Normalization** <br/>
We normalize the values using MinMax in order to have the same scale on all attributes

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(df_performances.values)

**Silhouette Viewer**
The Silhouette Viewer displays the silhouette coefficient for each sample on a per cluster basis, visually evaluating the density and separation between clusters.

In the SilhouetteVisualizer charts, clusters with higher scores have wider shapes, but less cohesive clusters will be below the average score across all clusters, which is plotted as a vertical dashed red line.

This is particularly useful for determining cluster imbalance or for selecting a value for K by comparing multiple viewers.

In [ ]:
model = KMeans(n_clusters)
visualizer = SilhouetteVisualizer(model)

visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
# calculate the cluster
kmeans = KMeans(n_clusters, n_init=10, max_iter=100)
kmeans.fit(X)

In [ ]:
# print the unique labels identified by K-Means
np.unique(kmeans.labels_, return_counts=True)

In [ ]:
# print the number of items for each cluster
hist, bins = np.histogram(kmeans.labels_, bins=range(0, len(set(kmeans.labels_)) + 1))
dict(zip(bins, hist))

In [ ]:
# print the center of clusters
kmeans.cluster_centers_

In [ ]:
# Plot Clusters with Paiplot
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = kmeans.labels_
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

In [ ]:
# Plot Clusters with PairGrid and in evidence the centroids of the clusters
cleaned_dataframe = df_performances.copy()

centroids_inversed = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=cleaned_dataframe.columns.values)
cleaned_dataframe["label"] = kmeans.labels_.astype(str)
centroids_inversed["label"] = ["0 centroid", "1 centroid", "2 centroid", "3 centroid"]
full_ds = pd.concat([cleaned_dataframe, centroids_inversed], ignore_index=True)

g = sns.PairGrid(
                full_ds, 
                hue="label",
                hue_order=["0", "1", "2", "3", "0 centroid", "1 centroid", "2 centroid", "3 centroid"],
                palette=["b", "r", "y", "g", "b", "r", "y", "g"],
                hue_kws={
                    "s": [20, 20, 20, 20, 500, 500, 500, 500], 
                    "marker": ["o", "o", "o", "o", "*", "*", "*", "*"]}
)
g.map(plt.scatter, linewidth=1, edgecolor="w")
g.add_legend()



## DBscan

For the DBscan the dataset is normalized using 2 scalers (StandardScaler and MinMaxScaler) and both obtained dataset are analyzed. Then the parameters used by DBscan, eps (that is the distance where looking for the points) and and min number of point to create a cluster are analyzed to fins the best ones. So, the parameter are chosen with the analysis and a plot of clusters is done.

**Normalization** <br />
We normalize the values using StandardScaler in order to have the same scale on all attributes

In [ ]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(df_performances)
scaled_dataframe = pd.DataFrame( scaled_array, columns = df_performances.columns )

In [ ]:
scaled_dataframe.describe()

**example of clustering with fixed parameters eps and min points**

In [ ]:
dbscan = DBSCAN(eps=0.75, min_samples=5)
dbscan.fit(scaled_dataframe)

In [ ]:
labels = dbscan.labels_
np.unique(dbscan.labels_, return_counts=True)

with these parameters we obtain 7 clusters (plus the cluster of not classified points) and one of them has many points while the others have few points. The following picture shows the plot of clusters

In [ ]:
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = labels
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

**choice of parameters** <br>
To choose the best parameters more combination of eps and min samples are analyzed

In [ ]:
dist = pdist(X=scaled_dataframe, metric='euclidean')  # pair-wise distance: how every record is far from all others
dist = squareform(dist)                      # distance matrix given the vector dist

Calculate the distance of the K-th neighbour, in order to find an upper bound for the eps values to analyze

In [ ]:
kmin, kmax = 1, 50
kth_distances = {}
for k in range(kmin, kmax + 1): # initialize k lists
    kth_distances[k] = []

In [ ]:
for d in dist:
    # argsort returns the indexes that would sort d
    index_kth_distance = np.argsort(d)[k]
    for k in range(kmin, kmax + 1):
        # append to kth_distances[k] the value in d that corresponds
        # to the distance of the i-th point (record) from its k-th nn.
        # it's like: kth_distances[k].append(sorted_d[k])), but we get "sorted_d[k]" by d[indexes_to_sort_d[k]]
        kth_distances[k].append(d[index_kth_distance])

In [ ]:
plt.figure(figsize=(50, 20))
for k in kth_distances.keys():
    plt.plot(range(0, len(kth_distances[k])), sorted(kth_distances[k]))
    
plt.ylabel('dist from k-th neighbor (eps)', fontsize=25)
plt.xlabel('sorted distances', fontsize=25)
#plt.ylim(top=5)
plt.ylim(bottom=-0.25)
plt.tick_params(axis='both', which='major', labelsize=25)
plt.grid()
plt.show()

#### Grid search for eps and min_samples

A function to define metrics to evaluate the parameters is defined. The define metrics are 2: the number of clusters obtained with the parameters and the mean noise between noise points and the 6 points closer (6-NN).

In [ ]:
def get_metrics(eps, min_samples, dataset, iter_):
    # Fitting 
    dbscan_model_ = DBSCAN(eps = eps, min_samples = min_samples)
    dbscan_model_.fit(dataset)
    
    # Mean Noise Point Distance metric
    noise_indices = dbscan_model_.labels_ == -1
    
    if True in noise_indices:
        neighboors = NearestNeighbors(n_neighbors = 6).fit(dataset)
        distances, indices = neighboors.kneighbors(dataset)
        noise_distances = distances[noise_indices, 1:]
        noise_mean_distance = round(noise_distances.mean(), 3)
    else:
        noise_mean_distance = None
    
    # Number of found Clusters metric    
    number_of_clusters = len(set(dbscan_model_.labels_[dbscan_model_.labels_ >= 0]))
    
    #print("%3d | Tested with eps = %3s and min_samples = %3s | %5s %4s" % (i, eps, min_samples, str(noise_mean_distance), number_of_clusters))
    
    return(noise_mean_distance, number_of_clusters)

Definition of values that must be analyzed for eps and the min number of samples. Looking at the plot above the used threashold for eps is 3

In [ ]:
eps_to_test = [round(x, 3) for x in np.arange(0.5, 3, 0.2)] 
min_samples_to_test = np.arange(4, 30, 2)
print(eps_to_test)
print(min_samples_to_test)

Creation of dataframes that will contain the result of analysis (one for the mean noise and one for the number of clusters)

In [ ]:
# Dataframe per la metrica sulla distanza media dei noise points dai K punti più vicini
results_noise = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

# Dataframe per la metrica sul numero di cluster
results_clusters = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

Analysis of chosed values for the parameters

In [ ]:
#search
i = 0

for eps in eps_to_test:
    for min_samples in min_samples_to_test:
        i += 1
        # Calcolo le metriche
        noise_metric, cluster_metric  = get_metrics(eps, min_samples, scaled_dataframe, i)
        # Inserisco i risultati nei relativi dataframe
        results_noise.loc[eps, min_samples] = noise_metric
        results_clusters.loc[eps, min_samples] = cluster_metric

Plot of the results of the analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8) )

sns.heatmap(results_noise, annot = True, ax = ax1, cbar = False).set_title("Mean Noise Points Distance")
sns.heatmap(results_clusters, annot = True, ax = ax2, cbar = False).set_title("Number of clusters")

ax1.set_xlabel("min_samples")
ax2.set_xlabel("min_samples")
ax1.set_ylabel("eps")
ax2.set_ylabel("eps")

plt.tight_layout()
plt.show()

#### choice of parameters 

Analyzing the tables we choose eps=1.1 and min_samples=16 to obtain 3 clusters plus the one for unclassified points.

In [ ]:
best_dbscan = DBSCAN(eps = 1.1, min_samples = 16)
# Fitting
best_dbscan.fit(scaled_dataframe)

labels = best_dbscan.labels_
np.unique(best_dbscan.labels_, return_counts=True)

Plot of the new clusters

In [ ]:
# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = best_dbscan.labels_
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

**Normalization** <br />
We normalize the values using MinMaxScaler in order to have the same scale on all attributes

In [ ]:
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(df_performances)
scaled_dataframe = pd.DataFrame( scaled_array, columns = df_performances.columns )

**example of clustering with fixed parameters eps and min points**

In [ ]:
dbscan = DBSCAN(eps=0.75, min_samples=5)
dbscan.fit(scaled_dataframe)
labels = dbscan.labels_
np.unique(dbscan.labels_, return_counts=True)

**choice of parameters** <br>
To choose the best parameters more combination of eps and min samples are analyzed

In [ ]:
dist = pdist(X=scaled_dataframe, metric='euclidean')  # pair-wise distance: how every record is far from all others
dist = squareform(dist)                      # distance matrix given the vector dist

Calculate the distance of the K-th neighbour, in order to find an upper bound for the eps values to analyze

In [ ]:
kmin, kmax = 3, 50
kth_distances = {}
for k in range(kmin, kmax + 1): # initialize k lists
    kth_distances[k] = []

In [ ]:
for d in dist:
    # argsort returns the indexes that would sort d
    index_kth_distance = np.argsort(d)[k]
    for k in range(kmin, kmax + 1):
        # append to kth_distances[k] the value in d that corresponds
        # to the distance of the i-th point (record) from its k-th nn.
        # it's like: kth_distances[k].append(sorted_d[k])), but we get "sorted_d[k]" by d[indexes_to_sort_d[k]]
        kth_distances[k].append(d[index_kth_distance])

In [ ]:
plt.figure(figsize=(50, 20))
for k in kth_distances.keys():
    plt.plot(range(0, len(kth_distances[k])), sorted(kth_distances[k]))
    
plt.ylabel('dist from k-th neighbor (eps)', fontsize=25)
plt.xlabel('sorted distances', fontsize=25)
#plt.ylim(top=5)
plt.ylim(bottom=-0.25)
plt.tick_params(axis='both', which='major', labelsize=25)
plt.grid()
plt.show()

Definition of values that must be analyzed for eps and the min number of samples. Looking at the plot above the used threashold for eps is 0.75

In [ ]:
eps_to_test = [round(x, 3) for x in np.arange(0.1, 0.75, 0.1)]
min_samples_to_test = np.arange(2, 16, 1)
print(eps_to_test)
print(min_samples_to_test)

Creation of dataframes that will contain the result of analysis (one for the mean noise and one for the number of clusters)

In [ ]:
# Dataframe per la metrica sulla distanza media dei noise points dai K punti più vicini
results_noise = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

# Dataframe per la metrica sul numero di cluster
results_clusters = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

Analysis of chosed values for the parameters

In [ ]:
i = 0

for eps in eps_to_test:
    for min_samples in min_samples_to_test:
        i += 1
        # Calcolo le metriche
        noise_metric, cluster_metric  = get_metrics(eps, min_samples, scaled_dataframe, i)
        # Inserisco i risultati nei relativi dataframe
        results_noise.loc[eps, min_samples] = noise_metric
        results_clusters.loc[eps, min_samples] = cluster_metric

Plot of the results of the analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4) )

sns.heatmap(results_noise, annot = True, ax = ax1, cbar = False).set_title("Mean Noise Points Distance")
sns.heatmap(results_clusters, annot = True, ax = ax2, cbar = False).set_title("Number of clusters")

ax1.set_xlabel("min_samples")
ax2.set_xlabel("min_samples")
ax1.set_ylabel("eps")
ax2.set_ylabel("eps")

plt.tight_layout()
plt.show()

#### choice of parameters 

Analyzing the tables we choose eps=0.1 and min_samples=9 to obtain 4 clusters plus the one for unclassified points.

In [ ]:
best_dbscan = DBSCAN(eps = 0.1, min_samples = 9)
# Fitting
best_dbscan.fit(scaled_dataframe)

labels = best_dbscan.labels_
np.unique(best_dbscan.labels_, return_counts=True)

Plot of the new clusters

In [ ]:
# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = best_dbscan.labels_
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

### Results

Analizyng wih both scalers we obtain a big cluster for unclassified points. With **StandardScaler**, this cluster has more than 400 points. About other clusters, one of them has many points (about 800) but the others have less then 20 points, so we can notice that clusters are very unbalanced. With **MinMaxScaler** the cluster of unclassified points is  even bigger with about 980 points. But some of the other clusters are relatively big (about 100 and 150 points), and some are small. So with this scaler there is not only a big cluster very big respect to the others but there are more bigger clusters and the difference between dimension of all clusters (except the unclassified one) is smaller.
To choose different parameters does not help to obtain better clusters. The results are not as clearly describable.

# Hierachical

In this section we will see hierarchical clustering performed the divisive technique, in particular, using the four methods to compute the distances between clusters. To compute the distances has been used euclidean distance.

In [ ]:
def count_cluster_elements(data, threshold, criterion='distance'):
    count = {}
    clusters = fcluster(data, threshold, criterion)
    for c in clusters:
        count[c] = count[c]+1 if c in count else 1
    return count, clusters

In [ ]:
methods = ['single', 'complete', 'average', 'ward']
models = {}

Using scipy we can compute the linkage matrix and use it for the dendrogram plot. This is done for all the criteria.

- ‘**single**’ uses the minimum of the distances between all observations of the two sets.

- ‘**complete**’ or ‘**maximum**’ linkage uses the maximum distances between all observations of the two sets.

- ‘**average**’ uses the average of the distances of each observation of the two sets.

- ‘**ward**’ minimizes the variance of the clusters being merged.

**Normalization** <br />
We normalize the values using MinMaxScaler in order to have the same scale on all attributes

In [ ]:
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(df_performances)
scaled_dataframe = pd.DataFrame( scaled_array, columns = df_performances.columns )

In [ ]:
for method in methods:
    link = linkage(scaled_dataframe, method=method, metric = 'euclidean')
    models[method] = link

Now plot the four dendrograms.

In [ ]:
f, axs = plt.subplots(ncols=4, figsize=(32,7))
i = 0
for model in models.keys():
    axs[i].set_title('Hierarchical Clustering by ' + model + ' Algorithm (' + methods[i] + '-linkage)')
    axs[i].set_xlabel('Players or (Cluster Size)')
    axs[i].set_ylabel('Distance')
    dend = dendrogram(models[model],ax=axs[i],truncate_mode='lastp', p=25, leaf_rotation=60, leaf_font_size = 8, show_contracted=True)
    i+=1

Using this function we can identify the longest uninterrupted vertical segment of the dendrogram.

In [ ]:
def compute_best_cut(model, scaled_dataframe):
    store_n_clusters = []
    store_distances = []
    step = 0.05
    n_clusters = 0
    i = 0

    while n_clusters != 1:
        distance_threshold = i*step
        cluster = AgglomerativeClustering(distance_threshold=distance_threshold, n_clusters=None, affinity='euclidean', linkage=model)
        cluster.fit_predict(scaled_dataframe)
        i = i + 1

        n_clusters = cluster.n_clusters_
        store_distances.append(distance_threshold)
        store_n_clusters.append(n_clusters)

    df = pd.DataFrame()
    df['distance_thresholds'] = store_distances
    df['n_clusters'] = store_n_clusters
    df_n_clusters_groups = df.groupby('n_clusters').size()
    n_for_opt_cut = df_n_clusters_groups.agg(['idxmax'])[0]
    distance = df[df.n_clusters == n_for_opt_cut].distance_thresholds.min()
    
    return n_for_opt_cut, distance

In [ ]:
cut_thresholds = {}
for model in models.keys():
    n_for_opt_cut, distance = compute_best_cut(model, scaled_dataframe)
    cut_thresholds[model] = distance
print(cut_thresholds)

In [ ]:
f, axs = plt.subplots(ncols=4, figsize=(32,7))
i = 0
for model in models.keys():
    axs[i].set_title('Hierarchical Clustering by ' + model + ' Algorithm (' + methods[i] + '-linkage)')
    axs[i].set_xlabel('Players or (Cluster Size)')
    axs[i].set_ylabel('Distance')
    axs[i].axhline(y=cut_thresholds[model], color="black")
    dend = dendrogram(models[model],ax=axs[i],truncate_mode='lastp', p=25, leaf_rotation=60, leaf_font_size = 8, show_contracted=True)
    i+=1


**Clusters evalution**

Let's compute some metrics to evaluate the clustering quality.

 **Silhouette coefficient**: defines how good the clusters are sperated. The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.
 
 
 **Davies-Bouldin index**: means the average ‘similarity’ between clusters, where the similarity is a measure that compares the distance between clusters with the size of the clusters themselves. Values closer to zero indicate a better partition.
 
 
 **Calinski and Harabasz score**: defined as ratio between the within-cluster dispersion and the between-cluster dispersion.

In [ ]:
model_labels = {}
metrics = ['silhouette', 'davies_bouldin', 'calinski_harabasz']
evaluation_metrics = pd.DataFrame(index=methods, columns=metrics)
for model in models.keys():
    print(model)
    count, clusters = count_cluster_elements(models[model], cut_thresholds[model])
    print(count)
    
    cluster = AgglomerativeClustering(n_clusters=len(count.keys()), affinity='euclidean', linkage=model)
    cluster.fit_predict(scaled_dataframe)
    labels = cluster.labels_
    model_labels[model] = labels
    silhouette = silhouette_score(scaled_dataframe, labels, metric='euclidean')
    davies_bouldin = davies_bouldin_score(scaled_dataframe, labels)
    calinski_harabasz = calinski_harabasz_score(scaled_dataframe, labels)
    evaluation_metrics.loc[model][metrics[0]] = silhouette
    evaluation_metrics.loc[model][metrics[1]] = davies_bouldin
    evaluation_metrics.loc[model][metrics[2]] = calinski_harabasz
evaluation_metrics

In the first part of the outputis showed the number of items for each cluster.

All three matrics show that the clustering has high difficulty to define well separated groups. Using the *single* metric the results improve, this is due to high similarity between the records, but are far away from a good clustering.

In [ ]:
i = 0
f, axs = plt.subplots(ncols=4, figsize=(32,7), sharey=True)
for model in models.keys():
    axs[i].set_title('Hierarchical Clustering by ' + model + ' Algorithm (' + methods[i] + '-linkage)')
    axs[i].scatter(df_performances['nmatch'].values, df_performances['best_rank_points'].values, c=model_labels[model] , s=25, cmap='viridis')
    axs[i].set_xlabel('nmatch')
    axs[i].set_ylabel('best_rank_points')
    i+=1

# XMEANS

In [ ]:
amount_initial_centers = 1   #number of clusters at the start. xmeans starts with only one cluster
max_n_clusters = 20

initial_centers = kmeans_plusplus_initializer(df_performances, amount_initial_centers).initialize()
xmeans_instance = xmeans(df_performances, initial_centers, kmax=max_n_clusters)
xmeans_instance.process(); #split with bayesian Information Criterion

clusters = xmeans_instance.get_clusters();
centers = xmeans_instance.get_centers()

print([len(c) for c in clusters])   

In [ ]:
# display allocated clusters
visualizer = cluster_visualizer_multidim();
visualizer.append_clusters(clusters, df_performances.values.tolist())
visualizer.append_cluster(centers, None, marker = '*', markersize=5)
#visualizer.show()
visualizer.show(pair_filter=[[0, 1], [0, 2], [0, 3]])

**plot with sns library**

In [ ]:
labels = np.zeros(df_performances.shape[0],  dtype=int) #num of rows
for i in range(len(clusters)):#number of cluster
    for j in clusters[i]: #index of row of dataset in cluster i
        labels[j] = int(i)

In [ ]:
palette_n = sns.color_palette("hls", n_colors=len(clusters))
palette_n

In [ ]:
# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = labels
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = palette_n)

# G-Means clustering algorithm

<a href="https://pyclustering.github.io/docs/0.10.1/html/dc/d86/namespacepyclustering_1_1cluster_1_1gmeans.html">G-MEANS by PYCLUSTERING</a>
<br />
<i>Greg Hamerly and Charles Elkan. Learning the k in k-means. In Proceedings of the 16th International Conference on Neural Information Processing Systems, NIPS'03, pages 281–288, Cambridge, MA, USA, 2003. MIT Press.</i>
<br /><br />
The G-means algorithm starts with a small number of centers, and grows the number of centers. Each iteration of the G-Means algorithm splits into two those centers whose data appear not to come from a Gaussian distribution. G means repeatedly makes decisions based on a statistical test for the data assigned to each center.


In [ ]:
data = df_performances.values.tolist()

 # Create instance of G-Means algorithm. By default the algorithm starts search from a single cluster.
max_n_clusters = 20
gmeans_instance = gmeans(data, k_max=max_n_clusters).process()
 
# Extract clustering results: clusters and their centers
clusters = gmeans_instance.get_clusters()
centers = gmeans_instance.get_centers()
 
# Print total sum of metric errors
print("Total WCE:", gmeans_instance.get_total_wce())

print([len(c) for c in clusters]) 

In [ ]:
# visualize a sample of obtained clusters
visualizer = cluster_visualizer_multidim()
visualizer.append_clusters(clusters, data = df_performances.values.tolist())
visualizer.show(pair_filter=[[0, 1], [0, 2], [0, 3]])

In [ ]:
# Print palette colors for clusters
palette_n = sns.color_palette("hls", n_colors=len(clusters))
palette_n

In [ ]:
# Prepare labels
labels = np.zeros(df_performances.shape[0],  dtype=int) #num of rows
for i in range(len(clusters)):#number of cluster
    for j in clusters[i]: #index of row of dataset in cluster i
        labels[j] = int(i)

# Print 
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = labels
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = palette_n)